# Train Set

In [ ]:
import os
import os.path as op
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
!git clone 'https://github.com/PakinDioxide/Dog-Breed-Classification.git'

In [ ]:
path = '/content/Dog-Breed-Classification/images/'

In [ ]:
!pip install -q fastbook

In [ ]:
from fastbook import *

In [ ]:
fields = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=Resize(224),
    batch_tfms=aug_transforms()
)

In [ ]:
dls = fields.dataloaders(f"{path}train", bs=64)

In [ ]:
dls.vocab

In [ ]:
dls.train.show_batch()

In [ ]:
learner = vision_learner(dls, resnet50, metrics=[error_rate, accuracy])

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(epochs=10, base_lr=0.0010000000474974513, freeze_epochs=1)

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.print_classification_report()

In [ ]:
interp.plot_confusion_matrix(figsize=(50,50))

In [ ]:
interp.most_confused(min_val=5)

In [ ]:
learner.show_results(shuffle=True)

In [ ]:
learner.export("dbc_resnet50_new_fastai.pkl")

# Test Set Prediction

In [ ]:
!git clone 'https://github.com/PakinDioxide/Dog-Breed-Classification.git'

In [ ]:
!pip install -q fastbook

In [ ]:
from fastbook import *
from google.colab import files
from IPython.display import Image
from IPython.display import HTML

In [ ]:
dblock2 = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #x - image; y - single class
    get_items=get_image_files, #get image
    splitter=GrandparentSplitter(valid_name='test'), #use parent folder as train-test split
    get_y=parent_label, #use parent folder as label
    item_tfms=Resize(224)
    )
dls2 = dblock2.dataloaders('/content/Dog-Breed-Classification/images/', bs=64) #batch size = 64

In [ ]:
dls2.vocab

In [ ]:
import torch

# Load the exported FastAI .pkl model
pkl_model_path = '/content/Dog-Breed-Classification/models/dbc_resnet50_new_fastai.pkl'
fastai_learner = torch.load(pkl_model_path, map_location=torch.device('cpu'))

# Extract the PyTorch model from the FastAI Learner
pytorch_model = fastai_learner.model

# Save the PyTorch model in .pth format
pth_model_path = '/content/Dog-Breed-Classification/models/dbc_resnet50_new_fastai.pth'
torch.save(pytorch_model.state_dict(), pth_model_path)


In [ ]:
learn = cnn_learner(dls2, resnet50, metrics=error_rate)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn.load('/content/Dog-Breed-Classification/models/dbc_resnet50_new_fastai'))
interp.plot_confusion_matrix(figsize=(50,50))

In [ ]:
interp.print_classification_report()

# Single Image Prediction

In [ ]:
!pip install fastbook

In [ ]:
from fastbook import *
from PIL import Image

In [ ]:
learn = load_learner('/content/dbc_resnet50_new_fastai.pkl')

In [ ]:
img = Image.open('/content/image.png')
img.resize([224, 224])

In [ ]:
learn.predict(img)